## Models to run

500 Hz 12 lead 
100 Hz 12 lead 
500 Hz 1 lead 

In [1]:
import os
import numpy as np
import pandas as pd

os.chdir("../..")
import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 500
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

  5%|▍         | 982/21799 [00:21<09:34, 36.21it/s]

KeyboardInterrupt: 

  5%|▍         | 982/21799 [00:40<09:34, 36.21it/s]

In [ ]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train_lead1 = X_train[:,:,0]
X_test_lead1 = X_test[:,:,0]
X_val_lead1 = X_val[:,:,0]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential

In [ ]:
lenet_5_model=Sequential()
lenet_5_model.add(Conv1D(filters=6, kernel_size=3, padding='same', input_shape=(1000,1)))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(Conv1D(filters=16, strides=1, kernel_size=5))
lenet_5_model.add(BatchNormalization())
lenet_5_model.add(Activation('relu'))
lenet_5_model.add(MaxPool1D(pool_size=2, strides=2, padding='same'))
lenet_5_model.add(GlobalAveragePooling1D())
lenet_5_model.add(Dense(64, activation='relu'))
lenet_5_model.add(Dense(32, activation='relu'))
lenet_5_model.add(Dense(n_classes, activation = 'sigmoid'))

In [ ]:
lenet_5_model.summary()

In [ ]:
lenet_5_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.AUC(num_thresholds=1000000, multi_label=False),
                utils.CustomMetric(beta=0.5)]
)

In [ ]:
history = lenet_5_model.fit(X_train_lead1, y_train, validation_data=(X_val_lead1, y_val), epochs=75)

In [ ]:
from sklearn.metrics import roc_auc_score

preds = lenet_5_model.predict(X_test_lead1)

print(roc_auc_score(y_test,preds, average='samples'))